## 1. Intro : Breaking RSA encryption and Order Finding

*In this case, we investigate **how RSA encryption (classical asymmetric cryptography) can be broken using quantum computing**, specially with **Shor's algorithm**.*

- RSA relies on the difficulty of factoring n = p × q
- If we can find the order of a mod n, we can factor n
- Goal: compare classical and quantum methods for order finding


# Python Imports

In [ ]:
from math import gcd
from fractions import Fraction
import itertools
import pandas as pd
import numpy as np
import time

# Quantum Programming
import qiskit as qs
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.circuit.library import UnitaryGate
from qiskit.circuit.library import QFT
from qiskit.visualization import visualize_transition
from qiskit.quantum_info import Statevector

from IPython.display import display, clear_output, Markdown, HTML
import ipywidgets as widgets

# Plotting (for visualization)
from matplotlib import pyplot as plt

#plt.rc('text', usetex=True)
#plt.rc('text.latex', preamble=r  '\usepackage{braket}') #Use the bracket packege for \ket

## 2. Classical Order Finding (3-4 min)
We search for the smallest positive `r` such that `a^r ≡ 1 mod n`.
This value `r` is called the order of `a modulo n`. It is cruicial for breaking RSA using Shor's algorithm.
We try all values of `r` starting from  and stop when `(a^r) % n == 1`.
In this example, we try pair of numbers (a, n), so we're looking for the order of a mod n and see exponential complexity (include timing)

In [8]:
# Adapted code from lecture 14 solution
def classical_order_finding(a, n):
    assert gcd(a, n) == 1
    
    t0 = time.time()
    power = a
    r = 1      
    while power != 1:
        r += 1
        power = pow(power, r, n)
        
    t = time.time() - t0   
    return r, t

# Caution: Order size (= power) does not correlate directly with n 
# Some large n value still yield small r
# Below pair of numbers are picked to show how classical algorithm struggles
# when orders (iterations) get higher
# Normally more orders ≠ proportionally more run time

for a, n in [[2, 15], [2, 1019], [2, 153488707270219], [2, 3416178076769076332873], [2, 10057227701218939403237]]:
    r, t = classical_order_finding(a, n)
    print(f"Total run-time to find order {r} of {a} mod {n} = {t:.3f}s")


Total run-time to find order 4 of 2 mod 15 = 0.000s
Total run-time to find order 509 of 2 mod 1019 = 0.000s
Total run-time to find order 219749 of 2 mod 153488707270219 = 0.584s
Total run-time to find order 2799187 of 2 mod 3416178076769076332873 = 12.128s
Total run-time to find order 8934907 of 2 mod 10057227701218939403237 = 57.170s


In [ ]:
# Utility function adapted from solution code of lecture 14
# Use this to see the cycle (periodicity)
def get_modular_exponentiation_sequence (a, n, max_power=10):
    powers = np.arange(0, max_power)
    result = [pow(a, int(r), n) for r in powers]  # in quantum r = k
    output_df = pd.DataFrame({"sequence": result})
    output_df.index.name = "power"
    return output_df

# Example of order of 2 mod 15
get_modular_exponentiation_sequence(2, 15)
#order = df[df['sequence'] == 1].index[1] # skip power=0
#print("Classical order r =", order)

,sequence
power,
0,1
1,2
2,4
3,8
4,1
5,2
6,4
7,8
8,1


## 3. Quantum Order Finding with Shor's Algorithm

### 3.1 Overview of Shor's Algorithm
1. Choose a random a < n such that gcd(a, n) = 1
2. Use Quantum Phase Estimation (QPE) to find the order `r`
3. Use classical post-processing to extract factors


### 3.2 Circuit: Modular Multiplication Gate `U`
We define a unitary operator `U |b⟩ = |a * b mod n⟩`.
This gate is used in the QPE circuit to encode the periodicity.

In [ ]:
# Utility function to build U |b⟩ = |a * b mod n⟩
def modular_multiplication_gate_U(a, n , num_qubits):
    dimention = 2**num_qubits
    U = np.zeros((dimention, dimention))
    
    for b in range(n):
      ab_mod_n = (a * b) % n
      U[ab_mod_n, b] = 1
      
    # rest to keep unitary
    for i in range(n, dimention):
      U[i, i] = 1
      
    return UnitaryGate(U, label=f'Ux{a} mod {n}')
    

In [ ]:
# Placeholder: implement or load modular multiplication gate circuit

# def qpe_order_finding_circuit(a, n, t):
#     qc = QuantumCircuit(t + len(bin(n)) - 2, t)
#     # Step 1: Apply Hadamards on counting qubits
      # Step 2: Apply Controlled modular multiplication gates
      # Step 3: Inverse QFT
      # Step 4: Measure counting register
#     return qc
# qc.draw('mpl')
# visualize_transition(qc)

### 3.3 Quantum Phase Estimation
We apply QPE using the `U` gate to extract the phase, which relates to `r`.
- Circuit diagram
- Measurement outcome (ϕ ≈ s/r)
- Continued fraction → r

In [ ]:
# Placeholder: build and simulate QPE circuit using Qiskit
# def quantum_order_finding(a, n):
#     ...
# return estimated_order

### 3.4 Results and Visualization
Plot the measurement results and analyze the most probable output state.
- Output: quantum circuit measured state
- If possible: animation or histogram showing most probable states
- Comment on success/failure, e.g., “r = 4 found in 75% of runs”

In [ ]:
# Placeholder: visualize quantum measurement output (e.g., histogram)

## 4. Classical vs Quantum Order Finding
We summarize and compare efficiency, complexity, and results.
- Table: runtime / gate depth / success chance
- Plot if helpful
- Use visual to emphasize: quantum = exponential speedup

In [ ]:
# Optional: create comparison table or plot
# Data: runtime or iteration counts vs problem size

## 5. Conlusion: Limitation and Final Thoughts (2 min)
- RSA encryption is **theoretically breakable** using quantum computing via Shor's algorithm.
- Why can't we break real RSA yet?
  - Qubit count
  - Coherence time
  - Gate fidelity
- Mention quantum-resistant crypto (brief)
- Conclude: quantum is promising, but not yet at threat to real RSA